In [20]:
from env import config
import requests

In [21]:
MINDSDB_UN=config("MINDSDB_UN", default=None)# martyna@mindsdb.com
MINDSDB_PW=config("MINDSDB_PW", default=None) # xxxxxxxx
assert MINDSDB_UN is not None
assert MINDSDB_PW is not None


MINDSDB_BASE_URL = "https://cloud.mindsdb.com/api"

In [22]:
def get_mindsdb_session():
    session = requests.Session()
    session.post('https://cloud.mindsdb.com/cloud/login', json={
        'email': MINDSDB_UN,
        'password': MINDSDB_PW
    })
    return session

In [23]:
def mindsdb_query(session, sql_query):
    endpoint = "/sql/query"
    url = f"{MINDSDB_BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    return session.post(url, json={"query": sql_query}, headers=headers)

In [53]:

# ['flightDate',
#  'startingAirport',
#  'destinationAirport',
#  'isBasicEconomy',
#  'isRefundable',
#  'isNonStop',
#  'segmentsAirlineName',
#  'totalFare',
#  '__mdb_ts_previous_totalFare',
#  'select_data_query',
#  'when_data',
#  'totalFare_original',
#  'totalFare_confidence',
#  'totalFare_explain',
#  'totalFare_anomaly',
#  'totalFare_min',
#  'totalFare_max',
 
def predict_query(session, 
        flightDate = "2022-04-21", 
        startingAirport="SFO", 
        isNonStop=1, 
        destinationAirport="BOS",
        raw_request=False,
        ):
    sql_query = f"""
    SELECT m.flightDate, m.segmentsAirlineName, m.isNonStop, m.totalFare, m.totalFare_confidence FROM mindsdb.flight_price_predictor AS m
    JOIN ai_travel_agent.flight_prices AS t
    WHERE t.flightDate = "{flightDate}"
    AND t.startingAirport = "{startingAirport}"
    AND t.isNonStop = {isNonStop}
    AND t.destinationAirport = "{destinationAirport}";
    """
    response = mindsdb_query(session, sql_query)
    response.raise_for_status()
    if raw_request:
        return response
    data = response.json()
    columns = data.get("column_names")
    dataset = data.get('data')
    web_ready_data = [dict(zip(columns, row)) for row in dataset]
    return web_ready_data
        
    

In [25]:
session = get_mindsdb_session()
query_response = predict_query(session, 
        flightDate = "2022-04-21", 
        startingAirport="BOS", 
        isNonStop=1, 
        destinationAirport="SFO",
        raw_request=False,)
session.close()

In [26]:
query_response

[{'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'Alaska Airlines',
  'isNonStop': 1.0,
  'totalFare': 407.60097800046026,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'United',
  'isNonStop': 1.0,
  'totalFare': 437.70167433898956,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'JetBlue Airways',
  'isNonStop': 1.0,
  'totalFare': 636.7351950883466,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'American Airlines',
  'isNonStop': 1.0,
  'totalFare': 601.1875918728093,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'Delta',
  'isNonStop': 1.0,
  'totalFare': 389.00500090014,
  'totalFare_confidence': 0.9991}]

In [54]:
pref = dict(flightDate = "2022-04-21", 
        startingAirport="LAX", 
        isNonStop=1, 
        destinationAirport="JFK",)
query_response = predict_query(session, 
        **pref,)
options = query_response
rec_context = json.dumps({"options": options, "preferences": pref})
rec_context

'{"options": [{"flightDate": "2022-04-21 00:00:00.000000", "segmentsAirlineName": "JetBlue Airways", "isNonStop": 1.0, "totalFare": 414.99701251022907, "totalFare_confidence": 0.9991}, {"flightDate": "2022-04-21 00:00:00.000000", "segmentsAirlineName": "United", "isNonStop": 1.0, "totalFare": 481.75790609878334, "totalFare_confidence": 0.9991}, {"flightDate": "2022-04-21 00:00:00.000000", "segmentsAirlineName": "Delta", "isNonStop": 1.0, "totalFare": 423.995500019981, "totalFare_confidence": 0.9991}, {"flightDate": "2022-04-21 00:00:00.000000", "segmentsAirlineName": "American Airlines", "isNonStop": 1.0, "totalFare": 439.79413415008287, "totalFare_confidence": 0.9991}], "preferences": {"flightDate": "2022-04-21", "startingAirport": "LAX", "isNonStop": 1, "destinationAirport": "JFK"}}'

In [61]:
rec_sql_query = f"""SELECT answer
FROM ai_travel_agent
WHERE question='Respond with JSON only, what is the best flight option?'
AND context='{rec_context}';
"""

In [62]:
session = get_mindsdb_session()
rec_prediction_response = mindsdb_query(session, rec_sql_query)
session.close()

In [63]:
rec_prediction_response.json()

{'column_names': ['answer'],
 'context': {'db': 'mindsdb'},
 'data': [['{\n  "flightDate": "2022-04-21 00:00:00.000000",\n  "segmentsAirlineName": "JetBlue Airways",\n  "isNonStop": 1.0,\n  "totalFare": 414.99701251022907,\n  "totalFare_confidence": 0.9991\n}']],
 'type': 'table'}

In [64]:
def recommended_flight(session, 
        user_data = {},
        forecast_data = [],
        question="Respond with JSON only, what is the best flight option?",
        raw_request=False,
        **kwargs,
        ):
    context = {"options": forecast_data, "preferences": user_data}
    context_data = json.dumps(context)
    sql_query = f"""SELECT answer
    FROM ai_travel_agent
    WHERE question='{question}'
    AND context='{context_data}';
    """
    response = mindsdb_query(session, sql_query)
    response.raise_for_status()
    if raw_request:
        return response
    data = response.json()
    dataset = data.get('data')
    if dataset is None or data is None:
        return []
    if isinstance(dataset, list):
        sub_dataset = dataset[0]
        if isinstance(sub_dataset, list):
            return sub_dataset[0]
        return sub_dataset
    return dataset

In [66]:
json.loads(recommended_flight(session, user_data = pref,
        forecast_data =query_response))

{'flightDate': '2022-04-21 00:00:00.000000',
 'segmentsAirlineName': 'JetBlue Airways',
 'isNonStop': 1.0,
 'totalFare': 414.99701251022907,
 'totalFare_confidence': 0.9991}

In [30]:
# web_ready_data = [dict(zip(columns, row)) for row in dataset]
# web_ready_data